#Connect to Google Drive

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/

#Pip install requirements

In [ ]:
!pip install opencv-python numpy ipython roboflow torch pyyaml ultralytics

#Get the imports

In [ ]:
import os
import cv2
import numpy as np
import shutil

from IPython import display
from IPython.display import display, Image, clear_output

from roboflow import Roboflow
import torch
import yaml

#Get the Yolo model

In [ ]:
# Set HOME to get your current working directory
HOME = os.getcwd()

# Download the YOLO model
%cd {HOME}
!git clone https://github.com/ultralytics/ultralytics.git
%cd {HOME}/ultralytics
!pip install -e .

#Get the dataset from Roboflow

In [ ]:
# Making and moving into 'datasets' directory
os.makedirs(f'{HOME}/datasets', exist_ok=True)
%cd {HOME}/datasets

# Get dataset from roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="socUkpgBT4KYDhbiU16Z")
project = rf.workspace("osha-violations").project("osha")
dataset = project.version(1).download("yolov12")

#Convert the folders for use

In [ ]:
def get_next_folder(base_name):
    index = 1
    while os.path.exists(f'{base_name}-{index}'):
        index += 1
    return f'{base_name}-{index}'

train_folder_name = get_next_folder('train')
predict_folder_name = get_next_folder('predict')
val_folder_name = get_next_folder('val')

#Get folder path and check if gpu is available

In [ ]:
# Get gpu for training if it exists
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

folder_path = f'{HOME}/datasets/{train_folder_name}/{train_folder_name}'

# Check if the folder exists, then delete it
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' has been deleted.")

#Prepare YAML file

In [ ]:
# Get a YAML file ready
file_path = '../datasets/OSHA-1/data.yaml'

# Open YAML
with open(file_path, 'r') as file:
    data = yaml.safe_load(file)

# Modify the 'train' and 'val' paths
data['train'] = 'train/images'
data['val'] = 'valid/images'

# Save YAML
with open(file_path, 'w') as file:
    yaml.dump(data, file)

print("YAML file updated successfully!")
print({train_folder_name})

#Get latest version of ultralytics

In [ ]:
!pip install --upgrade ultralytics

#Get the YOLO12 Model

In [ ]:
from ultralytics import YOLO
model = YOLO(f'{HOME}/yolov12m.pt')

#Start training the model

In [ ]:
model.train(
   data=f'{HOME}/datasets/OSHA-1/data.yaml',  # Path to data.yaml
   epochs=60,  # Number of epochs
   imgsz=640,   # Image size
   batch=16,    # Batch size
   name=f'{train_folder_name}',  # Name for this training run
   project=f'{train_folder_name}',  # Save directory
   workers=12,   # Number of workers for data loading
   device=device,  # Set the correct device (MPS, CUDA, or CPU)
)

#Save the model

In [ ]:
# Save the model and the results
model = YOLO(f'{HOME}/datasets/{train_folder_name}/{train_folder_name}/weights/best.pt')
save_dir = os.path.join(f'{HOME}/ultralytics/runs/detect', val_folder_name)  # Changed from segment to detect
val_folder_name = get_next_folder('val')
model.val(data=f'{dataset.location}/data.yaml', save_dir=save_dir)

#Display results of training

In [ ]:
# Display the results for the training data
display(Image(filename=f"{HOME}/datasets/{train_folder_name}/{train_folder_name}/val_batch0_pred.jpg", width=600))